In [1]:
from pathlib import Path
import pickle
import time
from tqdm import tqdm
import math

from projectmir.xmldocument import XMLDocument
from projectmir.extract_definition import pattern_based_extract_description, pagel_ranking_candidates, kato_ranking_candidates, Definition

In [2]:
p_path = './data/test_latexml/'
p = Path(p_path)
x = list(p.glob('*.html'))
documents_IDs = [x_.name[:-5] for x_ in x]
docs_list = [[]]*len(documents_IDs)
for i, documents_ID in tqdm(enumerate(documents_IDs)):
    document_path = p_path + documents_ID + '.html'
    start_time = time.time()
    docs_list[i] = XMLDocument(document_path)
save_filename = 'docs_list' + time.strftime('_%d%b%Y_%H%M%S') + '.pkl'
with open(save_filename, 'wb') as f:
    pickle.dump(docs_list, f)

0it [00:00, ?it/s]2020-08-25 00:45:02 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-08-25 00:45:02 INFO: Use device: cpu
2020-08-25 00:45:02 INFO: Loading: tokenize
2020-08-25 00:45:02 INFO: Done loading processors!
2020-08-25 00:45:03 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-08-25 00:45:03 INFO: Use device: cpu
2020-08-25 00:45:03 INFO: Loading: tokenize
2020-08-25 00:45:03 INFO: Loading: pos
2020-08-25 00:45:04 INFO: Loading: lemma
2020-08-25 00:45:04 INFO: Loading: depparse
2020-08-25 00:45:04 INFO: Loading: ner
2020-08-25 00:45:05 INFO: Done loading processors!
1it [00:51, 51.47s/it]2020-08-25 00:45:54 INFO: Loading these models for language: en (English):
| Processor | Package |
------------------

loaded data.
processing data...
preprocessed document
extract identifiers...
Number of math components is 50
POS tagging...
extract candidate definition...
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-613df5e95a0f40c7.props -preload tokenize,ssplit,pos,lemma,ner,depparse
compute the properties of candidates...
elapsed time: 51.47154 seconds ---
loaded data.
processing data...
preprocessed document
extract identifiers...
Number of math components is 50
POS tagging...
extract candidate definition...
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp

In [3]:
load_filename = 'docs_list.pkl'
with open(load_filename, 'rb') as f:
    docs_list = pickle.load(f)
doc0 = docs_list[0]
print(doc0.path)


./data/test_latexml/test_sample.html


In [27]:
# extract definitions using three methods.
pattern_based_definition_list = []
pagel_definition_list = []
kato_definition_list = []
params = {'sigma_d': math.sqrt(12 / math.log(2)),
          'sigma_s': 2 / math.sqrt(math.log(2)),
          'alpha': 1,
          'beta': 1,
          'gamma': 0.1,
          'eta': 1}
print('id',
      'identifier',
      # 'pattern_based',
      'pagel_score',
      'pagel',
      'kato_score',
      'kato',
      sep='\t\t\t')
for i, identifier in enumerate(doc0.identifiers):
    definition_pattern_based = pattern_based_extract_description(identifier)
    if not definition_pattern_based:
        definition_pattern_based = [Definition(definition='')]
    pattern_based_definition_list.append(definition_pattern_based)

    definition_pagel = pagel_ranking_candidates(identifier, params)
    pagel_definition_list.append(definition_pagel)

    definition_kato = kato_ranking_candidates(identifier, params)
    kato_definition_list.append(definition_kato)


    print(identifier.text_tex,
          # definition_pattern_based[0].definition,
          f'{definition_pagel[0].score:.4f}',
          definition_pagel[0].definition,
          f'{definition_kato[0].score:.4f}',
          definition_kato[0].definition, sep='\t\t\t'
          )

id			identifier			pagel_score			pagel			kato_score			kato
w_i			0.9050			a mass flow rate			0.9262			an inlet temperature MATH0001
T_i			0.5452			a single component with a mass flow rate MATH0000 and an inlet temperature			0.6919			a single component with a mass flow rate MATH0000 and an inlet temperature
Q			0.5833			an electrical heater that provides a heating rate			0.3952			an electrical heater that provides a heating rate
T			0.5378			the exit temperature			0.3643			the exit temperature
w			0.5468			the liquid holdup MATH0005			0.3704			the liquid holdup MATH0005
V			0.5607			the liquid holdup			0.3798			the liquid holdup
ρ			0.3087			the liquid			0.2091			the liquid
C			0.5378			The density MATH0006 and heat capacity			0.3643			The density MATH0006 and heat capacity
U_int			0.6536			the enthalpy			0.5234			the internal energy
H			0.7928			the enthalpy			0.5370			the enthalpy
\overset{^}{U}_int			0.5833			MATH0009			0.6512			the caret ( ^ )
\overset{^}{H}			0.5312			MATH0010			0.6